# 线性规划和整数规划模型
 在工程技术、经济管理、科学研究、军事作战训练及日常生产生活等众多领域中，人们常常会遇到各种优化问题。例如，在生产经营中，我们总是希望制订最优的生产计划，充分利用已有的人力、物力资源，获得最大的经济效益；在运输问题中，我们总是希望设计最优的运输方案，在完成运输任务的前提下，力求运输成本最小等。针对优化问题的数学建模问题也是数学建模竞赛中一类比较常见的问题，这样的问题常常可以使用数学规划模型进行研究。

数学规划是运筹学的一个重要分支，而线性规划又是数学规划中的一部分主要内容。很多实际问题都可以归结为“线性规划”问题。线性规划（Linear Programming，简记为LP）有比较完善的理论基础和有效的求解方法，在实际问题中有极其广泛的应用。特别是随着计算机技术的飞速发展，线性规划的应用在深度和广度上有了极大的提高。

## 4.1 线性规划模型

### 4.1.1 线性规划模型及概念

#### 1.引例
某企业利用两种原材料A和B生产三种产品$P_1,P_2和P_3 。$已知每生产1公斤的产品所消耗的原材料A、B的数量（单位：公斤）和花费的加工时间C（单位：小时），每公斤产品销售后所带来的利润（单位：元）以及每天可用的资源的数量如表4.1所示，则该企业应该如何制订每天的生产计划，才能使所获利润达到最大？

问题分析： 该问题是在企业的生产经营中经常面临的一个问题：如何制订一个最优的生产计划？因为原材料和加工时间的可用数量是有限的，这也就构成了该问题的约束条件，而解决该问题也就是在满足上述约束条件的前提下，确定三种产品的产量，使得产品销售后所获得的利润达到最大值。

模型假设： 假设该企业的产品不存在积压，即产量等于销量。

符号说明： $设x_i表示产品P_i每天的产量,i = 1, 2, 3。通常称 为决策变量。$

#### 例4.1
**目标函数：**
$$
\max \ z = 70x_1 + 50x_2 + 60x_3
$$
**约束条件：**
$$
\begin{cases}
2x_1 + 4x_2 + 3x_3 \leq 150, \\
3x_1 + x_2 + 5x_3 \leq 160, \\
7x_1 + 3x_2 + 5x_3 \leq 200, \\
x_i \geq 0, \quad i = 1, 2, 3.
\end{cases}
$$

In [1]:
import cvxpy as cp
import numpy as np
c = np.array([70, 50, 60])
a = np.array([[2, 4, 3],
              [3, 1, 5],
              [7, 3, 5]])
b = np.array([150, 160, 200])
x = cp.Variable(3, pos = True)
ojb = cp.Maximize(c @ x)
cons = [a @ x <= b]
prob = cp.Problem(ojb, cons)
prob.solve()
print('最优解为：', x.value)
print('最优值为：', prob.value)

最优解为： [ 8.35184614 21.98820975 15.11448955]
最优值为： 2590.909090524814


#### 例题4.2
求解如下线性规划模型：
$\max z = 1.15x_{41} + 1.40x_{23} + 1.25x{32} + 1.06x_{54}$
$$
\begin{cases}
x_{11} + x_{14}  =  100000,\\
x_{21} + x_{23} + x_{24}  =  1.06x_{14},\\
x_{31} + x_{32} + x_{34}  =  1.15x_{11} + 1.06x_{24},\\
x_{41} + x_{44} = 1.15x_{21} + 1.06x_{34},\\
x_{54}  = 1.15x_{31} + 1.06x_{44},\\
x_{32} \leq 40000, x_{23} \leq 30000,\\
x_{ij} \geq 0, i = 1, 2, 3, 4, 5;j = 1, 2, 3, 4.\\
\end{cases}
$$

In [1]:
import cvxpy as cp
x = cp.Variable((5, 4), pos = True)
obj = cp.Maximize(1.15 * x[3, 0] + 1.40 * x[1, 2] + 1.25 * x[2, 1] + 1.06 * x[4, 3])
cons = [x[0, 0] + x[0, 3] == 100000,
        x[1, 0] + x[1, 2] + x[1, 3] == 1.06 * x[0, 3],
        x[2, 0] + x[2, 1] + x[2, 3] == 1.15 * x[0, 0] + 1.06 * x[1, 3],
        x[3, 0] + x[3, 3] == 1.15 * x[1, 0] + 1.06 * x[2, 3],
        x[4, 3] == 1.15 * x[2, 0] + 1.06 * x[3, 3],
        x[2, 1] <= 40000,
        x[1, 2] <= 30000]
prob = cp.Problem(obj, cons)
prob.solve(solver = 'GLPK_MI')
print('最优值为：', prob.value)
print('最优解为：', x.value)

最优值为： 143750.0
最优解为： [[34782.60869565     0.             0.         65217.39130435]
 [39130.43478261     0.         30000.             0.        ]
 [    0.         40000.             0.             0.        ]
 [45000.             0.             0.             0.        ]
 [    0.             0.             0.             0.        ]]


#### 例题4.3
使用Python软件计算6个产地8个销地的最小费用运输问题

| 单位运价,产地,销地	 | B1	 | B2	            | B3	          | B4	        | B5	      | B6	    | B7	  | B8 |	产量|
|-------------|-----|----------------|--------------|------------|----------|--------|------|----|---|
 | A1	         | 6	  | 2	  | 6	  | 7	  | 4	  | 2	  | 5	  | 9	  | 60  |
 | A2	         | 4	  | 9	  | 5	  | 3	  | 8	  | 5	  | 8	  | 2	  | 55  |
 | A3	         | 5	  | 2	  | 1	  | 9	  | 7	  | 4	  | 3	  | 3	  | 51               |
 | A4	         | 7	  | 6	| 7	| 3	| 9	| 2	| 7	| 1	 | 43  |
 | A5	         | 2	  | 3	             | 9	           | 5	         | 7	       | 2	     | 6	   | 5	 | 41               |
 | A6	         | 5	  | 5	             | 2	           | 2	         | 8	       | 1	     | 4	   | 3	 | 52  |
| 销量	         | 35	 | 37	            | 22	          | 32	        | 41	      | 32	    | 43	  | 38 |



In [6]:
import numpy as np
import cvxpy as cp
import pandas as pd
c=np.genfromtxt("data4_5_1.txt", dtype=float, max_rows=6, usecols=range(8)) #读前6行前8列数据
e=np.genfromtxt("data4_5_1.txt", dtype=float, max_rows=6, usecols=8) #读最后一列数据
d=np.genfromtxt("data4_5_1.txt", dtype=float, skip_header=6) #读最后一行数据
x=cp.Variable((6,8), pos=True)
obj=cp.Minimize(cp.sum(cp.multiply(c, x)))
con= [cp.sum(x, axis=0)==d,
      cp.sum(x, axis=1)<=e]
prob = cp.Problem(obj, con)
prob.solve(solver='GLPK_MI')
print("最优值为:", prob.value)
print("最优解为：\n", x.value)
xd=pd.DataFrame(x.value)
xd.to_excel("data4_5_2.xlsx")  #数据写到Excel文件，便于做表使用


最优值为: 664.0
最优解为：
 [[ 0. 19.  0.  0. 41.  0.  0.  0.]
 [ 0.  0.  0. 32.  0.  0.  0.  1.]
 [ 0. 12. 22.  0.  0.  0. 17.  0.]
 [ 0.  0.  0.  0.  0.  6.  0. 37.]
 [35.  6.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 26. 26.  0.]]


genfromtxt用法

        import numpy as np
        data = np.genfromtxt('filename.txt', delimiter=',')

- fname: 文件名或文件路径。可以是字符串、文件对象或生成器。
- delimiter: 用于分隔值的字符或字符串。默认是任何空白字符。
- dtype: 输出数组的数据类型。默认是 float。
- skip_header: 跳过文件开头的行数。
- skip_footer: 跳过文件末尾的行数。
- usecols: 指定要读取的列索引或列名。
- missing_values: 指定表示缺失值的字符串或字符串列表。
- filling_values: 指定用于替换缺失值的值。
- comments: 指定注释字符，默认是 #。
- autostrip: 是否自动去除字符串中的空白字符，默认是 False。

In [8]:
import numpy as np
import pandas as pd
data = pd.read_excel('data4_5_3.xlsx', header = None)
data = data.values
c = data[: -1, : -1]
d = data[-1, : -1]
e = data[:-1, -1]
x = cp.Variable((6,8), pos=True)
obj = cp.Minimize(cp.sum(cp.multiply(c, x)))
cons = [
    cp.sum(x, axis = 0) == d,
    cp.sum(x, axis = 1) <= e
]
prob = cp.Problem(obj, cons)
prob.solve(solver='GLPK_MI')
print("最优值为:", prob.value)
print("最优解为：\n", x.value)
xd=pd.DataFrame(x.value)
xd.to_excel("data4_5_4.xlsx")  #数据写到Excel文件，便于做表使用

最优值为: 664.0
最优解为：
 [[ 0. 19.  0.  0. 41.  0.  0.  0.]
 [ 0.  0.  0. 32.  0.  0.  0.  1.]
 [ 0. 12. 22.  0.  0.  0. 17.  0.]
 [ 0.  0.  0.  0.  0.  6.  0. 37.]
 [35.  6.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 26. 26.  0.]]
